In [3]:
import pandas as pd
import numpy as np
import seaborn as sns


In [4]:
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    """[summary]

    Args:
        df (pd.DataFrame): [description]

    Returns:
        pd.DataFrame: [description]
    """
    calculate_cols = [
        'id', 'tag', 'solver_id', 'benchmark_id', 'task_id', 'cut_off',
        'timed_out', 'exit_with_error', 'runtime', 'anon_1', 'benchmark_name',
        'symbol','validated','correct','instance'
    ]
    return (df[calculate_cols].rename(columns={
        "anon_1": "solver_full_name",
        "symbol": "task"
    }).astype({
        'solver_id': 'int16',
        'benchmark_id': 'int16',
        'task_id': 'int8',
        'cut_off': 'int16',
        'runtime': 'float32'
    }))

In [5]:
data = pd.read_csv("./data.csv")
data = prepare_data(data)
data

,id,tag,solver_id,benchmark_id,task_id,cut_off,timed_out,exit_with_error,runtime,solver_full_name,benchmark_name,task,validated,correct,instance
0,2293,val_test_2,1,5,7,2,False,False,0.008144,MY_SOLVER_2_1.4,validation_benchmark,SE-ST,True,correct,A-1-afinput_exp_acyclic_depvary_step3_batch_yy...
1,2294,val_test_2,1,5,7,2,False,False,0.006390,MY_SOLVER_2_1.4,validation_benchmark,SE-ST,True,correct,A-1-afinput_exp_acyclic_indvary1_step5_batch_y...
2,2295,val_test_2,1,5,7,2,False,False,0.006953,MY_SOLVER_2_1.4,validation_benchmark,SE-ST,True,correct,A-1-afinput_exp_acyclic_indvary1_step5_batch_y...
3,2296,val_test_2,1,5,7,2,False,False,0.008385,MY_SOLVER_2_1.4,validation_benchmark,SE-ST,True,correct,A-1-afinput_exp_acyclic_indvary3_step1_batch_y...
4,2297,val_test_2,1,5,7,2,False,False,0.772178,MY_SOLVER_2_1.4,validation_benchmark,SE-ST,True,correct,A-2-afinput_exp_acyclic_depvary_step6_batch_yy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,2856,val_test_2,4,5,10,2,False,False,0.025951,MatrixX_0.96,validation_benchmark,SE-CO,True,correct,n100p3q34ve.apx
564,2857,val_test_2,4,5,10,2,False,False,0.031389,MatrixX_0.96,validation_benchmark,SE-CO,True,correct,n128p5q2_ve.apx
565,2858,val_test_2,4,5,10,2,False,False,0.036895,MatrixX_0.96,validation_benchmark,SE-CO,True,correct,n160p5q2_ve.apx
566,2859,val_test_2,4,5,10,2,False,False,0.043753,MatrixX_0.96,validation_benchmark,SE-CO,True,correct,n192p5q2_ve.apx


In [12]:
def create_vbs(df,vbs_id):
    best_runtime = df['runtime'].min()
    row = df.iloc[0]
    row['solver_id'] = vbs_id
    row['solver_full_name'] = "vbs"
    row['id'] = 0
    row['runtime'] = best_runtime
    row['validated'] = True
    row['correct'] = 'correct'
    
    return row

In [21]:
grouping = ['tag', 'task_id', 'benchmark_id', 'instance']
vbs_id = data['solver_id'].max() + 1
vbs_df = data.groupby(grouping,as_index=False).apply(lambda df: create_vbs(df,vbs_id))

data.append(vbs_df)

data = data.groupby(grouping,as_index=False).apply(lambda df: df.assign(rank=df['runtime'].rank(method='dense',ascending=True)))
type(data)


Index(['id', 'tag', 'solver_id', 'benchmark_id', 'task_id', 'cut_off',
       'timed_out', 'exit_with_error', 'runtime', 'solver_full_name',
       'benchmark_name', 'task', 'validated', 'correct', 'instance', 'rank'],
      dtype='object')
Index(['id', 'tag', 'solver_id', 'benchmark_id', 'task_id', 'cut_off',
       'timed_out', 'exit_with_error', 'runtime', 'solver_full_name',
       'benchmark_name', 'task', 'validated', 'correct', 'instance', 'rank'],
      dtype='object')
Index(['id', 'tag', 'solver_id', 'benchmark_id', 'task_id', 'cut_off',
       'timed_out', 'exit_with_error', 'runtime', 'solver_full_name',
       'benchmark_name', 'task', 'validated', 'correct', 'instance', 'rank'],
      dtype='object')
Index(['id', 'tag', 'solver_id', 'benchmark_id', 'task_id', 'cut_off',
       'timed_out', 'exit_with_error', 'runtime', 'solver_full_name',
       'benchmark_name', 'task', 'validated', 'correct', 'instance', 'rank'],
      dtype='object')
Index(['id', 'tag', 'solver_id', 'be

id         tag  solver_id  benchmark_id  task_id  cut_off  \
0   0    2293  val_test_2          1             5        7        2   
    71   2364  val_test_2          2             5        7        2   
    142  2435  val_test_2          3             5        7        2   
    213  2506  val_test_2          4             5        7        2   
1   1    2294  val_test_2          1             5        7        2   
...       ...         ...        ...           ...      ...      ...   
140 566  2859  val_test_2          4             5       10        2   
141 354  2647  val_test_2          1             5       10        2   
    425  2718  val_test_2          2             5       10        2   
    496  2789  val_test_2          3             5       10        2   
    567  2860  val_test_2          4             5       10        2   

         timed_out  exit_with_error   runtime solver_full_name  \
0   0        False            False  0.008144  MY_SOLVER_2_1.4   
    71       False            False  0.006617  MY_SOLVER_3_1.4   
    142      False            False  0.007492    ArgSemSAT_1.4   
    213      False            False  0.025674     MatrixX_0.96   
1   1        False            False  0.006390  MY_SOLVER_2_1.4   
...            ...              ...       ...              ...   
140 566      False            False  0.043753     MatrixX_0.96   
141 354      False            False  0.017970  MY_SOLVER_2_1.4   
    425      False            False  0.025109  MY_SOLVER_3_1.4   
    496       True            False       NaN    ArgSemSAT_1.4   
    567      False            False  0.053588     MatrixX_0.96   

               benchmark_name   task  validated  correct  \
0   0    validation_benchmark  SE-ST       True  correct   
    71   validation_benchmark  SE-ST       True  correct   
    142  validation_benchmark  SE-ST       True  correct   
    213  validation_benchmark  SE-ST       True  correct   
1   1    validation_benchmark  SE-ST       True  correct   
...                       ...    ...        ...      ...   
140 566  validation_benchmark  SE-CO       True  correct   
141 354  validation_benchmark  SE-CO       True  correct   
    425  validation_benchmark  SE-CO       True  correct   
    496  validation_benchmark  SE-CO      False      NaN   
    567  validation_benchmark  SE-CO       True  correct   

                                                  instance  rank  
0   0    A-1-afinput_exp_acyclic_depvary_step3_batch_yy...   3.0  
    71   A-1-afinput_exp_acyclic_depvary_step3_batch_yy...   1.0  
    142  A-1-afinput_exp_acyclic_depvary_step3_batch_yy...   2.0  
    213  A-1-afinput_exp_acyclic_depvary_step3_batch_yy...   4.0  
1   1    A-1-afinput_exp_acyclic_indvary1_step5_batch_y...   3.0  
...                                                    ...   ...  
140 566                                    n192p5q2_ve.apx   3.0  
141 354                                    n224p5q2_ve.apx   1.0  
    425                                    n224p5q2_ve.apx   2.0  
    496                                    n224p5q2_ve.apx   NaN  
    567                                    n224p5q2_ve.apx   3.0  

[568 rows x 16 columns]